In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap
%matplotlib inline

In [ ]:
data = pd.read_csv('THREADS.results.csv')
data.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
data.loc[:, 'Benchmark'] = data['Benchmark'].str.replace('.x', '')

average_data = data.groupby(['Benchmark','NumThreads', 'Success', 'RunId'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
average_data.head()

In [ ]:
benchmarks = data["Benchmark"].unique()
print(benchmarks)

In [ ]:
def one_thread_relative_data(average_data, benchmark):
    one_thread_data = average_data.loc[average_data["NumThreads"] == 1]
    
    relative_data = average_data.copy(deep=True)
    
    for benchmark in benchmarks:
        benchmark_one_thread_data = one_thread_data.loc[relative_data["Benchmark"] == benchmark]
        energy = benchmark_one_thread_data.iloc[0]["Energy"]
        time = benchmark_one_thread_data.iloc[0]["Time"]

        relative_data.loc[relative_data["Benchmark"] == benchmark, "Energy"] /= energy
        relative_data.loc[relative_data["Benchmark"] == benchmark, "Time"] /= time

    return relative_data

def double_plot_relative_to_one_thread(relative_data, benchmark):
    benchmark_data = relative_data[relative_data["Benchmark"] == benchmark]
    
    X = benchmark_data["NumThreads"].values
    
    energy = benchmark_data["Energy"].values
    time = benchmark_data["Time"].values
    

    plt.figure()
    plt.scatter(X, energy, marker='o', label='Energy')
    plt.scatter(X, time, marker='x', label='Time')
    
    plt.axhline(1, label='1 Thread')
    
    plt.title('\n'.join(wrap(f'Best improvement of Energy and Time relative to 1 Thread for {benchmark}.', 55)))
    plt.ylabel('Relative to -1 Thread')
    plt.xlabel('Number of threads"')
    plt.legend()

In [ ]:
relative_data = one_thread_relative_data(average_data, benchmarks)

for benchmark in benchmarks:
    double_plot_relative_to_one_thread(relative_data, benchmark)

In [ ]:
def plot_energy_vs_time(benchmark, data):
    benchmark_data = data.loc[data["Benchmark"] == benchmark]

    plt.figure(figsize=(12, 10))
    
    for num_threads in benchmark_data["NumThreads"].unique():
        thread_data = benchmark_data[benchmark_data["NumThreads"] == num_threads]
        
        X = thread_data["Time"].values
        Y = thread_data["Energy"].values

        plt.scatter(X, Y, label=f"{num_threads} threads")

    plt.title('Energy vs Time results for different thread counts on ' + benchmark)
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')

In [ ]:
for benchmark in benchmarks:
    plot_energy_vs_time(benchmark, data)